# **Authentication**

In [ ]:
import tweepy
from textblob import TextBlob



consumer_key ='C3n0EIBkRhpM6EohRja4wBxEH'
consumer_key_secret ='Uu127IPY1R8yQT2klE4WAD4qsK9DKDaahptvSpDOq2gkib9pth'
access_token ='954075172264431616-v8ldAac4R4XJguYwja6nVhY0zzP25Wi'
access_token_secret='5JXnXe3YHD4FC5gC6USU4rtRNsif7Xp6wTarE7uYHF6zP'


# Priyanks access ->
# consumer_key = 'ZqvGZgMWusmV8cVYekYAzD1CJ'
# consumer_key_secret = 'iR2QptF7WfMlrdFdtCPPEmlyQWdx4qEMUd6IvtpArXLGmIqETe'
# access_token = '1602361115349917697-Gw1Fx4hEqSBGlM8kx0eXMwf8nytpfg'
# access_token_secret = 'IfsH8uSVFqWCvEUwtMHH9zR54vBl3oF0JRXthLH9Vtfo9'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


# **Fetch the real time Data**

In [ ]:
def search(query, count):
    df_tweets = pd.DataFrame(columns = ['username','retweetcount', 'tweet'])
    try:
      tweets_list = [tweet for tweet in tweepy.Cursor(api.search, q=query, lang="en", tweet_mode='extended').items(count)]
      for tweet in tweets_list:
          username = tweet.user.screen_name
          retweetcount = tweet.retweet_count
          try:
              tweet = tweet.retweeted_status.full_text
          except AttributeError:  
              tweet = tweet.full_text
          df_tweets.loc[len(df_tweets)] = [username, retweetcount, tweet]
    except tweepy.TweepError as e:
          print(f"Error: Twitter Authentication Failed - \n{str(e)}") 
    return df_tweets

In [ ]:
df = search("elon", 800)

Error: Twitter Authentication Failed - 
Twitter error response: status code = 401


In [ ]:
df.info()
df.head()

# **Data Preprocessing**

In [ ]:
!pip install tweet-preprocessor

In [ ]:

df.info()

df=df.drop_duplicates(subset=['tweet'], keep=False)

df.info()


In [ ]:
import preprocessor as p
# URL,  Mentions are removed
df['clean_tweet1'] = df['tweet'].apply(lambda x : p.clean(x))
df.head()


In [ ]:
df['clean_tweet2'] = df['clean_tweet1'].str.replace("[^a-zA-Z# ]", '' ,regex=True )
#punctuations are removed
df.head()

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = set(stopwords.words("english"))

#remove stop words
df['most_clean_tweet'] = df['clean_tweet2'].apply(lambda x: ' '.join([item for item in x.split() for j in item.split() if j not in stopWords ]))

df.head()

clean_tweet3 was the most processed tweet.



In [ ]:

df['polarity'] = df['most_clean_tweet'].apply(lambda x : TextBlob(x).sentiment.polarity)

df['Sentiment'] = df['polarity'].apply(lambda x : 'negative' if x < 0 else 'positive' if x >0 else 'neutral')

df.head()

# **Model**

In [ ]:
target_variable = df['Sentiment'].apply(lambda x: 0 if x=='negative' else 1 if x=='positive' else -1 )

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# bag-of-words vectorised matrix
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words='english', max_features=150)
bow_word_feature = bow_vectorizer.fit_transform(df['most_clean_tweet'])

#get_feature_names_out provides the uniques words of the vocabulary
print(bow_vectorizer.get_feature_names_out())

#frequency of each word within the document
bow_word_feature.todense()

X_trainBW, X_testBW, y_trainBW, y_testBW = train_test_split(bow_word_feature, target_variable, test_size=0.25, random_state=3342)


In [ ]:

from sklearn.feature_extraction.text import  TfidfVectorizer
# Tfid vectorised matrix

Tfid_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, stop_words='english', max_features=150)
Tfid_feature = Tfid_vectorizer.fit_transform(df['most_clean_tweet'])

print(Tfid_vectorizer.get_feature_names_out())
Tfid_feature.todense()

X_trainTF, X_testTF, y_trainTF, y_testTF = train_test_split(Tfid_feature, target_variable, test_size=0.25, random_state=3342)


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


def processModel(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train.toarray(), y_train)
    # predictions
    predictions = classifier.predict(X_test.toarray())
    # calculating accuracy
    accuracy=metrics.accuracy_score(predictions,y_test)
    return accuracy;

In [ ]:

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)

DecisionTreeClassifier_BOW = processModel(classifier, X_trainBW, X_testBW, y_trainBW, y_testBW)*100

print('DecisionTreeClassifier_BOW Accuracy Score : ', DecisionTreeClassifier_BOW)

DecisionTreeClassifier_TF_IDF = processModel(classifier, X_trainTF, X_testTF, y_trainTF, y_testTF )*100

print('DecisionTreeClassifier_TF_IDF Accuracy Score : ', DecisionTreeClassifier_TF_IDF)


In [ ]:

classifier = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', random_state = 0)

RandomForestClassifier_BOW = processModel(classifier, X_trainBW, X_testBW, y_trainBW, y_testBW)*100

print('RandomForestClassifier_BOW Accuracy Score : ', RandomForestClassifier_BOW)

RandomForestClassifier_TF_IDF = processModel(classifier, X_trainTF, X_testTF, y_trainTF, y_testTF )*100

print('RandomForestClassifier_TF_IDF Accuracy Score : ', RandomForestClassifier_TF_IDF)

In [ ]:
naive_classifier = GaussianNB()

GuassionNaiveBayes_BOW = processModel(naive_classifier, X_trainBW, X_testBW, y_trainBW, y_testBW)*100

print('GuassionNaiveBayes_BOW Accuracy Score : ', GuassionNaiveBayes_BOW)

GuassionNaiveBayes_TF_IDF = processModel(naive_classifier, X_trainTF, X_testTF, y_trainTF, y_testTF )*100

print('GuassionNaiveBayes_TF_IDF Accuracy Score : ', GuassionNaiveBayes_TF_IDF)

# **Comparision**




In [ ]:
import matplotlib.pyplot as plt

DecisionTreeClassifierData = {'DecisionTreeClassifier - BOW ': DecisionTreeClassifier_BOW, 'DecisionTreeClassifier - TF_IDF': DecisionTreeClassifier_TF_IDF}

RandomForestClassifierData = {'RandomForestClassifier - BOW': RandomForestClassifier_BOW, 'RandomForestClassifier - TF_IDF': RandomForestClassifier_TF_IDF}
                
NaiveBayesData = { 'Naive Bayes - BOW':GuassionNaiveBayes_BOW, 'Naive Bayes - TF_IDF':GuassionNaiveBayes_TF_IDF }


fig = plt.figure(figsize = (20, 8))
 
plt.bar(list(DecisionTreeClassifierData.keys()), list(DecisionTreeClassifierData.values()), color ='maroon', width = 0.5)
plt.bar(list(RandomForestClassifierData.keys()), list(RandomForestClassifierData.values()), color ='royalblue', width = 0.5)
plt.bar(list(NaiveBayesData.keys()), list(NaiveBayesData.values()), color ='mediumseagreen', width = 0.5)
 
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Accuracy of models")
plt.show()